## new testing section for the notebook


## Testing to use the message from 3 commas to start and stop the trading bot

In [2]:
import requests

# Define the API endpoint
api_url = "https://3commas.io/trade_signal/trading_view"

# Your data payload this is for stopping the bot
payload = {
    "action": "stop_bot",
    "message_type": "bot",
    "bot_id": 11767904,
    "email_token": "910eaf15-60fd-4e5b-ac35-a22125a38efe",
    "delay_seconds": 0
}

# # Your data payload this is for starting the bot
# payload = {
#     "action": "start_bot",  
#     "message_type": "bot",  
#     "bot_id": 11767904,  
#     "email_token": "910eaf15-60fd-4e5b-ac35-a22125a38efe",  
#     "delay_seconds": 0
# }

# Send POST request
response = requests.post(api_url, json=payload)

# Check response
if response.status_code == 200:
    print("Bot stopped successfully.")
else:
    print("Failed to stop the bot. Status code:", response.status_code)

Bot stopped successfully.


In [9]:
#
# todo convert a string dict to a dictionary

str_dict = '''{
    "action": "start_bot",  
    "message_type": "bot",  
    "bot_id": 11767904,  
    "email_token": "910eaf15-60fd-4e5b-ac35-a22125a38efe",  
    "delay_seconds": 0 
}'''

print(type(str_dict))

# Convert string dict to dict
trade_dict = eval(str_dict)
print(type(trade_dict))
import json
trade_dict = json.dumps(trade_dict, indent=4)
print(trade_dict)


<class 'str'>
<class 'dict'>
{
    "action": "start_bot",
    "message_type": "bot",
    "bot_id": 11767904,
    "email_token": "910eaf15-60fd-4e5b-ac35-a22125a38efe",
    "delay_seconds": 0
}


In [150]:
from datetime import datetime, timedelta
import pandas as pd

symbol = 'BTC-USDC'
period = '1day'

# note VIP0 is 2000 requests per 30 seconds
# Type of candlestick patterns: 1min, 3min, 5min, 15min, 30min, 1hour, 2hour, 4hour, 6hour, 8hour, 12hour, 1day, 1week
# get todays date with year month and day only include hour minute and second
end = datetime.now().strftime("%Y %m %d %H %M %S")
# start 2 years ago
start = (datetime.now() - timedelta(days=730)).strftime("%Y %m %d %H %M %S")
# convert end to timestamp
end = datetime.strptime(end, "%Y %m %d %H %M %S").timestamp()
# convert start to timestamp
start = datetime.strptime(start, "%Y %m %d %H %M %S").timestamp()
# API endpoint
url = f'https://api.kucoin.com/api/v1/market/candles?type={period}&symbol={symbol}&startAt={int(start)}&endAt={int(end)}'

# Making a GET request
response = requests.get(url)

# Checking if the request was successful
if response.status_code == 200:
    # Parsing response data
    data = response.json()
    print("Crypto Data received by Kucoin:", data.keys())
    # create a dataframe from the json data
    df = pd.DataFrame(data['data'])
    # rename the columns
    df.columns = ['Date', 'Open', 'Close', 'High', 'Low',  'Volume', 'Turnover']
    # convert date column to an integar
    df['Date'] = df['Date'].astype(int)
    # convert the timestamp to datetime
    df['Date'] = pd.to_datetime(df['Date'], unit='s')
    # set the index to be the date column
    df.set_index('Date', inplace=True)
    # set the columns to be floats
    df = df.astype(float) 
    # sort the dataframe by date
    df = df.sort_index(ascending=True)
    # keep only the columns in this order Open High Low Close
    df = df[['Open', 'High', 'Low', 'Close']]

Crypto Data received by Kucoin: dict_keys(['code', 'data'])


In [151]:
df

,Open,High,Low,Close
Date,,,,
2021-12-06,49453.1,50959.3,47000.0,50545.6
2021-12-07,50517.0,51969.5,50000.1,50632.8
2021-12-08,50650.0,51188.7,48088.0,50527.0
2021-12-09,50548.4,50828.0,46300.1,47610.8
2021-12-10,47644.1,50126.5,46871.3,47069.2
...,...,...,...,...
2023-12-01,37726.0,39036.7,37618.4,38671.0
2023-12-02,38712.2,39751.7,38660.1,39457.3
2023-12-03,39467.2,40248.2,39302.7,39986.2


In [ ]:
#
#TODO: CREATE CLASSES FOR THE 3 METHOD SOF GATHERING DATA AND PREDICTING AND RETURNING THE FIGURE